## Full Text Search / Information Retrieval 

## 1) : Why build an Inverted Index for text-retrieval?

## 2) : Objectives:
* Collect a corpus
* Perform a brute-force query and measure the performance
* Create a forward and inverted index from the above 
* Search on one or a combination of terms and measure performance

In [1]:
from tqdm import tqdm
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from collections import defaultdict
import time
import re

In [2]:
corpus = fetch_20newsgroups(remove=['header', 'footer'])['data']

---

### Brute Force

#### Lets start with a example for one term query

In [14]:
def brute_force_query(query, corpus): 
    """Returns indices of matching documents"""
    doc_ids = []
    for i, doc in enumerate(corpus):
        for word in re.findall('(?u)\\b\\w\\w+\\b', doc.lower()):
        #for word in doc.lower().split():
            if word == query:
                doc_ids.append(i)
    return doc_ids

In [16]:
brute_force_query('bricklin', corpus)

[0, 958, 2835, 4988]

2.37 s ± 431 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Ok lets measure the performance of this approach

In [18]:
%timeit brute_force_query('bricklin', corpus)

2.74 s ± 85.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


---

### Create a forward and an inverted Index 

In [19]:
def create_forward_index(corpus):
    forward_index = defaultdict(list)
    for i, doc in enumerate(corpus):
        forward_index[i] = re.findall('(?u)\\b\\w\\w+\\b',doc.lower()) #recognise this regex anyone?
    return forward_index

In [20]:
def create_inverted_index(corpus):
    index = defaultdict(list) 
    for i, doc in enumerate(corpus):
        for word in re.findall('(?u)\\b\\w\\w+\\b',doc.lower()):
            # index.setdefault(word, [])
            index[word].append(i)
    return index

#### Ok lets measure the performance of this approach

In [21]:
result = create_inverted_index(corpus)

In [24]:
%timeit create_inverted_index(corpus)

1.55 s ± 379 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Is this what we expected?

In [25]:
%timeit result['bricklin']

73.7 ns ± 1.41 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


---

### Rank the documents which contain the most references

In [26]:
def find_terms(query, index):
    query = query.lower().split()
    results = defaultdict(list) 
    for term in query:
        results[term] = index[term]
    return results

In [27]:
find_terms('bricklin car', result)

defaultdict(list,
            {'bricklin': [],
             'car': [0,
              0,
              0,
              17,
              17,
              17,
              17,
              17,
              17,
              17,
              29,
              30,
              30,
              30,
              30,
              30,
              48,
              56,
              64,
              72,
              77,
              84,
              156,
              156,
              156,
              156,
              156,
              262,
              262,
              262,
              274,
              292,
              491,
              491,
              492,
              492,
              526,
              526,
              564,
              565,
              571,
              571,
              571,
              596,
              596,
              596,
              598,
              621,
              621,
              631,
              655,
  

---

### Other things to consider:

* Results ranking isn't as simple as finding all the results!!
    * e.g - which documents should come back first requires some page ranking - cf Markov and his buddies!!
    * e.g - intra-doc ranking - titles/links could be more important than words in text
    * e.g - intra-doc ranking - long documents will have more occurances of a term
* standard NLP: tokenization, stop words, language, remove crap, parsing, encoding
* typos: Levenshtein distance "hippo" - "hypo" -> Levenshtein dist == 2
* trigram search (heuristic algorithm)

---

### Technologies that utilize inverted-index from scratch:

* Postgres
* Mongodb
* Elasticsearch

---